In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.preprocessing import MinMaxScaler

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    roc_auc_score,
    roc_curve,
    precision_recall_curve,
    average_precision_score,
    confusion_matrix,
    brier_score_loss
)
from sklearn.calibration import calibration_curve


XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Users/justinwilliams/Desktop/School/semester 4/System Analysis And Design/sports_analysis/ml-workshop/training/.venv/lib/python3.10/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <89AD948E-E564-3266-867D-7AF89D6488F0> /Users/justinwilliams/Desktop/School/semester 4/System Analysis And Design/sports_analysis/ml-workshop/training/.venv/lib/python3.10/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/lib

# Outline: 
- EDA
- Feature Importance
- PSI?
- Feature Engineering?
- Feature Selection
- Visualizations?
- Modeling
  - Train Test Split Cross Validation
  - Hyperparameter tuning
- Evaluate model
- Save model

In [ ]:
from xgboost import XGBClassifier

dataset = pd.read_csv('data.csv')

target_col = 'Bankrupt?'

dataset

In [ ]:
dataset[target_col].value_counts()

In [ ]:
# stratification

In [ ]:
dataset[target_col].value_counts()/2

In [ ]:
dataset[target_col].shape

In [ ]:
## This is an important number. It reflects the overall default rate across the dataset 
dataset[target_col].mean()

In [ ]:
# Return a list of columns which don't contain the target_col
possible_features = dataset.columns.difference([target_col])
len(possible_features)

In [ ]:
# Filtering X as your features
# y contains only your target column
X = dataset[possible_features]
y = dataset[target_col]


In [ ]:
type(y)

In [ ]:
len(
    
)

In [ ]:
ex = pd.DataFrame(
    {'x1': [i for i in range(1,30)],
     'x2': [i* 4 for i in range(1,30)]
    }
)

In [ ]:
ex.corr()

In [ ]:
import numpy as np

corr_matrix = X.corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [col for col in upper_tri.columns if any(upper_tri[col] > 0.7)]
X = X.drop(columns=to_drop)
print("Removed features:", to_drop)


In [ ]:
# Splitting data for feature importance analysis
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Define parameters
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'max_depth': 6,
    'eta': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8
}

model = xgb.train(params, dtrain, num_boost_round=100)

# Measures the average improvement in model accuracy each time a feature is used in a split.
feature_importance = pd.DataFrame({
    'Feature': model.get_score(importance_type="gain").keys(),
    'Importance': model.get_score(importance_type="gain").values()}
)
feature_importance = feature_importance.sort_values(by="Importance", ascending=False)

In [ ]:
NUM_FEATS = 30

plt.figure(figsize=(10, 6))
sns.barplot(x=feature_importance.Importance[:NUM_FEATS], y=feature_importance.Feature[:NUM_FEATS])
plt.xlabel("Importance Score")
plt.ylabel("Feature")
plt.title("Top 10 Feature Importances")
plt.show()

cat_model = CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, loss_function='Logloss', verbose=0)
cat_model.fit(X_train, y_train)


cat_feature_importance = cat_model.get_feature_importance()
cat_feature_names = X_train.columns


cat_importance_df = pd.DataFrame({'Feature': cat_feature_names, 'Importance': cat_feature_importance})
cat_importance_df = cat_importance_df.sort_values(by='Importance', ascending=False)

NUM_FEATS = 30
plt.figure(figsize=(10, 6))
sns.barplot(x=cat_importance_df.Importance[:NUM_FEATS], y=cat_importance_df.Feature[:NUM_FEATS])
plt.xlabel("Importance Score")
plt.ylabel("Feature")

In [ ]:
importance_df = pd.merge(cat_importance_df,feature_importance, on='Feature', suffixes=('_cat','_xgb'))

# Normalize using MinMaxScaler
scaler = MinMaxScaler()
importance_df[['Importance_cat', 'Importance_xgb']] = scaler.fit_transform(
    importance_df[['Importance_cat', 'Importance_xgb']]
)

importance_df['Avg_Importance'] = importance_df[['Importance_cat', 'Importance_xgb']].mean(axis=1)

importance_df = importance_df.sort_values(by="Avg_Importance", ascending=False)

plt.figure(figsize=(12, 6))
sns.barplot(x=importance_df.Avg_Importance[:NUM_FEATS], y=importance_df.Feature[:NUM_FEATS])
plt.xlabel("Average Importance Score")
plt.ylabel("Feature")
plt.title("Top 10 Most Important Features (XGBoost & CatBoost Combined)")
plt.show()

In [ ]:
NUM_TOP_FEATURES = 30  # Change as needed
top_features = importance_df['Feature'].head(NUM_TOP_FEATURES).tolist()

X_top = X[top_features]

X_train, X_temp, y_train, y_temp = train_test_split(
    X_top, y, test_size=0.4, stratify=y, random_state=42
)
X_test, X_oos, y_test, y_oos = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

In [ ]:
# Train an XGBoost classifier
# THIS IS NOT DOING HYPERPARAMETER TUNING
model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    n_estimators=100,
    use_label_encoder=False,
    random_state=42
)
model.fit(X_train, y_train)

In [ ]:
import warnings
warnings.filterwarnings("ignore")


In [ ]:
# Method 2:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

param_dist = {
    'max_depth': [3, 4, 5, 6, 7, 8],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'n_estimators': [50, 100, 150, 200]
}

xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42
)

random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_dist,
    n_iter=20,
    scoring='roc_auc',
    cv=5,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)
print("Best parameters:", random_search.best_params_)
print("Best ROC AUC score:", random_search.best_score_)

model = random_search.best_estimator_


In [ ]:
# Evaluate on the Test set
y_test_proba = model.predict_proba(X_test)[:, 1]
y_test_pred = model.predict(X_test)

In [ ]:
roc_auc_test = roc_auc_score(y_test, y_test_proba)
pr_auc_test = average_precision_score(y_test, y_test_proba)
brier_test = brier_score_loss(y_test, y_test_proba)
cm_test = confusion_matrix(y_test, y_test_pred)

print("Test set evaluation:")
print("ROC AUC:", roc_auc_test)
print("PR AUC:", pr_auc_test)
print("Brier Score:", brier_test)
print("Confusion Matrix:\n", cm_test)

# ROC AUC plot (Test)
fpr, tpr, _ = roc_curve(y_test, y_test_proba)
plt.figure(figsize=(6, 4))
plt.plot(fpr, tpr, label=f'ROC (AUC = {roc_auc_test:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve (Test)")
plt.legend()
plt.show()

# Precision-Recall curve (Test)
precision, recall, _ = precision_recall_curve(y_test, y_test_proba)
plt.figure(figsize=(6, 4))
plt.plot(recall, precision, label=f'PR (AUC = {pr_auc_test:.2f})')
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve (Test)")
plt.legend()
plt.show()

# Calibration plot (Test)
prob_true, prob_pred = calibration_curve(y_test, y_test_proba, n_bins=10)
plt.figure(figsize=(6, 4))
plt.plot(prob_pred, prob_true, marker='o', linewidth=1, label='Calibration curve')
plt.plot([0, 1], [0, 1], linestyle='--', label='Perfect Calibration')
plt.xlabel("Mean predicted probability")
plt.ylabel("Fraction of positives")
plt.title("Calibration Plot (Test)")
plt.legend()
plt.show()

# Evaluate on the OOS set (similar metrics)
y_oos_proba = model.predict_proba(X_oos)[:, 1]
y_oos_pred = model.predict(X_oos)

roc_auc_oos = roc_auc_score(y_oos, y_oos_proba)
pr_auc_oos = average_precision_score(y_oos, y_oos_proba)
brier_oos = brier_score_loss(y_oos, y_oos_proba)
cm_oos = confusion_matrix(y_oos, y_oos_pred)

print("\nOOS set evaluation:")
print("ROC AUC:", roc_auc_oos)
print("PR AUC:", pr_auc_oos)
print("Brier Score:", brier_oos)
print("Confusion Matrix:\n", cm_oos)

# ROC AUC plot (OOS)
fpr, tpr, _ = roc_curve(y_oos, y_oos_proba)
plt.figure(figsize=(6, 4))
plt.plot(fpr, tpr, label=f'ROC (AUC = {roc_auc_oos:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve (OOS)")
plt.legend()
plt.show()

# Calibration plot (OOS)
prob_true, prob_pred = calibration_curve(y_oos, y_oos_proba, n_bins=10)
plt.figure(figsize=(6, 4))
plt.plot(prob_pred, prob_true, marker='o', linewidth=1, label='Calibration curve')
plt.plot([0, 1], [0, 1], linestyle='--', label='Perfect Calibration')
plt.xlabel("Mean predicted probability")
plt.ylabel("Fraction of positives")
plt.title("Calibration Plot (OOS)")
plt.legend()
plt.show()

In [ ]:
model.save_model("model.json")


In [ ]:
model.feature_names_in_

In [26]:
# Test inference

In [39]:
X_oos

,Net Value Growth Rate,Equity to Liability,Borrowing dependency,Interest-bearing debt interest rate,Total debt/Total net worth,Non-industry income and expenditure/revenue,After-tax net Interest Rate,Total income/Total expense,Interest Coverage Ratio (Interest expense to EBIT),Degree of Financial Leverage (DFL),...,Total Asset Growth Rate,Cash flow rate,No-credit Interval,Inventory/Working Capital,Net Income to Total Assets,Operating Profit Growth Rate,Inventory Turnover Rate (times),Cash/Total Assets,Revenue per person,Operating Expense Rate
1534,0.000871,0.021482,0.379579,0.000658,0.011730,0.302385,0.808718,0.002083,0.563721,0.026557,...,1.090870e-04,0.462693,0.623175,0.276896,0.764491,0.848138,1.016509e-04,0.048216,0.003152,4.409220e-04
4090,0.000437,0.027259,0.376996,0.000723,0.007704,0.303518,0.809316,0.002246,0.566110,0.027037,...,4.910000e+09,0.466214,0.623833,0.277351,0.798904,0.848004,9.400000e+09,0.018707,0.051693,9.760000e+09
3534,0.000663,0.044983,0.369882,0.000413,0.003706,0.303532,0.809530,0.002484,0.565177,0.026795,...,1.300256e-04,0.472409,0.624048,0.277149,0.858368,0.849787,9.000000e+09,0.430226,0.021474,2.195937e-04
1413,0.000414,0.054653,0.371565,0.000743,0.002870,0.303117,0.808956,0.002067,0.564855,0.026731,...,6.080000e+09,0.470220,0.623733,0.277487,0.751339,0.847790,2.600000e+09,0.034456,0.003992,3.393931e-04
6549,0.000424,0.046342,0.369637,0.000000,0.003562,0.303448,0.809230,0.002175,0.565158,0.026791,...,6.330000e+09,0.449935,0.624398,0.276975,0.784835,0.845154,1.910000e+07,0.260775,0.005567,2.694330e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6455,0.000457,0.024997,0.377157,0.000150,0.008905,0.303460,0.809318,0.002243,0.566754,0.027281,...,7.390000e+09,0.458942,0.623664,0.277553,0.797761,0.848005,3.630000e+09,0.072903,0.021021,1.761144e-04
6548,0.000479,0.047904,0.370232,0.000205,0.003409,0.303674,0.809427,0.002503,0.565192,0.026798,...,6.650000e+09,0.466293,0.624410,0.277062,0.830716,0.848017,2.206893e-04,0.393126,0.019749,9.660000e+09
5692,0.000573,0.031303,0.371651,0.000279,0.006200,0.303656,0.809441,0.002763,0.565228,0.026806,...,9.850000e+08,0.470033,0.625038,0.277046,0.840692,0.847999,3.429761e-04,0.239749,0.120421,4.730000e+09
3205,0.000480,0.025414,0.379195,0.000454,0.008657,0.303287,0.809233,0.002155,0.559376,0.026226,...,6.740000e+09,0.470636,0.623761,0.276916,0.790782,0.848124,1.949900e-03,0.066300,0.005158,1.135550e-04


In [23]:
X_oos.head(10).to_csv('inference_test.csv')

In [34]:
model = xgb.Booster()
model.load_model("model.json")

In [35]:
final_features = [
    " Net Value Growth Rate",
    " Equity to Liability",
    " Borrowing dependency",
    " Interest-bearing debt interest rate",
    " Total debt/Total net worth",
    " Non-industry income and expenditure/revenue",
    " After-tax net Interest Rate",
    " Total income/Total expense",
    " Interest Coverage Ratio (Interest expense to EBIT)",
    " Degree of Financial Leverage (DFL)",
    " Cash Turnover Rate",
    " Quick Ratio",
    " Quick Assets/Current Liability",
    " Current Liability to Current Assets",
    " Operating profit per person",
    " Net Value Per Share (A)",
    " Research and development expense rate",
    " Allocation rate per person",
    " Quick Asset Turnover Rate",
    " Average Collection Days",
    " Total Asset Growth Rate",
    " Cash flow rate",
    " No-credit Interval",
    " Inventory/Working Capital",
    " Net Income to Total Assets",
    " Operating Profit Growth Rate",
    " Inventory Turnover Rate (times)",
    " Cash/Total Assets",
    " Revenue per person",
    " Operating Expense Rate",
]

In [36]:
df = pd.read_csv('inference_test.csv')
print(df.columns)

missing = set(final_features) - set(df.columns)
if missing:
    print({"error": f"Missing columns in CSV: {missing}"})

df_final = df[final_features]
predictions = model.predict(df_final)


Index(['Unnamed: 0', ' Net Value Growth Rate', ' Equity to Liability',
       ' Borrowing dependency', ' Interest-bearing debt interest rate',
       ' Total debt/Total net worth',
       ' Non-industry income and expenditure/revenue',
       ' After-tax net Interest Rate', ' Total income/Total expense',
       ' Interest Coverage Ratio (Interest expense to EBIT)',
       ' Degree of Financial Leverage (DFL)', ' Cash Turnover Rate',
       ' Quick Ratio', ' Quick Assets/Current Liability',
       ' Current Liability to Current Assets', ' Operating profit per person',
       ' Net Value Per Share (A)', ' Research and development expense rate',
       ' Allocation rate per person', ' Quick Asset Turnover Rate',
       ' Average Collection Days', ' Total Asset Growth Rate',
       ' Cash flow rate', ' No-credit Interval', ' Inventory/Working Capital',
       ' Net Income to Total Assets', ' Operating Profit Growth Rate',
       ' Inventory Turnover Rate (times)', ' Cash/Total Assets',
   

TypeError: ('Expecting data to be a DMatrix object, got: ', <class 'pandas.core.frame.DataFrame'>)

In [38]:
model = XGBClassifier()
model.load_model("model.json")

df = pd.read_csv('inference_test.csv')
print(df.columns)

missing = set(final_features) - set(df.columns)
if missing:
    print({"error": f"Missing columns in CSV: {missing}"})

df_final = df[final_features]
predictions = model.predict(df_final)


Index(['Unnamed: 0', ' Net Value Growth Rate', ' Equity to Liability',
       ' Borrowing dependency', ' Interest-bearing debt interest rate',
       ' Total debt/Total net worth',
       ' Non-industry income and expenditure/revenue',
       ' After-tax net Interest Rate', ' Total income/Total expense',
       ' Interest Coverage Ratio (Interest expense to EBIT)',
       ' Degree of Financial Leverage (DFL)', ' Cash Turnover Rate',
       ' Quick Ratio', ' Quick Assets/Current Liability',
       ' Current Liability to Current Assets', ' Operating profit per person',
       ' Net Value Per Share (A)', ' Research and development expense rate',
       ' Allocation rate per person', ' Quick Asset Turnover Rate',
       ' Average Collection Days', ' Total Asset Growth Rate',
       ' Cash flow rate', ' No-credit Interval', ' Inventory/Working Capital',
       ' Net Income to Total Assets', ' Operating Profit Growth Rate',
       ' Inventory Turnover Rate (times)', ' Cash/Total Assets',
   